## Feature Engineering Part 2

1. gather the data, split, scale
2. feature selection
3. linear regression

In [27]:
import pandas as pd
import numpy as np
import wrangle
import warnings
warnings.filterwarnings("ignore")

In [28]:
path='../../data/student-mat.csv'

df, X_train_explore, \
    X_train_scaled, y_train, \
    X_validate_scaled, y_validate, \
    X_test_scaled, y_test = wrangle.wrangle_student_math(path)

In [29]:
from sklearn.feature_selection import SelectKBest, f_regression

In [30]:
f_selector = SelectKBest(f_regression, 5)
f_selector.fit(X_train_scaled, y_train)
f_mask = f_selector.get_support()
X_train_scaled_f_reduced = X_train_scaled.iloc[:,f_mask]
X_train_scaled_f_reduced.head(1)

,Medu,Fedu,failures,G1,G2
142,1.0,1.0,0.0,0.357143,0.578947


In [37]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled_f_reduced, y_train)
y_pred = lm.predict(X_train_scaled_f_reduced)

preds = pd.DataFrame({'y_pred': y_pred, 'y': y_train})

from sklearn.metrics import mean_squared_error
import math
# rmse
math.sqrt(mean_squared_error(preds.y_pred, preds.y))

1.8505870357921905

In [38]:
# compare that to rfe gives us

from sklearn.feature_selection import RFE
# creat3 the ML model object
lm = LinearRegression()
# create the rfe object
rfe = RFE(lm, 5)
# fit the rfe
rfe.fit(X_train_scaled, y_train)
# get the feature support boolean list
rfe_mask = rfe.support_
# reduce the dataframe to just those features
X_train_scaled_rfe_reduced = X_train_scaled.iloc[:,rfe_mask]

X_train_scaled_rfe_reduced.head(1)

,age,famrel,absences,G1,G2
142,0.0,0.75,0.035714,0.357143,0.578947


In [39]:
lm = LinearRegression()
lm.fit(X_train_scaled_rfe_reduced, y_train)
# predict y using X
y_pred_rfe = lm.predict(X_train_scaled_rfe_reduced)

# create dataframe of predictions with actual
preds_rfe = pd.DataFrame({'y_pred_rfe': y_pred_rfe, 'y': y_train})

# compute the rmse
math.sqrt(mean_squared_error(preds_rfe.y_pred_rfe, preds_rfe.y))

1.7872885302885915

## The wrangle function

In [ ]:
def wrangle_student_math(path):
    df = pd.read_csv(path, sep=";")
    
    # drop any nulls
    df = df[~df.isnull()]

    # get object column names
    object_cols = get_object_cols(df)
    
    # create dummy vars
    df = create_dummies(df, object_cols)
      
    # split data 
    X_train, y_train, X_validate, y_validate, X_test, y_test = train_validate_test(df, 'G3')
    
    # get numeric column names
    numeric_cols = get_numeric_X_cols(X_train, object_cols)

    # scale data 
    X_train_scaled, X_validate_scaled, X_test_scaled = min_max_scale(X_train, X_validate, X_test, numeric_cols)
    
    return df, X_train, X_train_scaled, y_train, X_validate_scaled, y_validate, X_test_scaled, y_test

Run each of those individually and see what you get! 

In [ ]:
path='../../data/student-mat.csv'
df = pd.read_csv(path, sep=";")
    
# drop any nulls
df = df[~df.isnull()]

# get object column names
object_cols = wrangle.get_object_cols(df)

# create dummy vars
df = wrangle.create_dummies(df, object_cols)
      
# split data 
X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.train_validate_test(df, 'G3')
    
# get numeric column names
numeric_cols = wrangle.get_numeric_X_cols(X_train, object_cols)

# scale data 
X_train_scaled, X_validate_scaled, X_test_scaled = wrangle.min_max_scale(X_train, X_validate, X_test, numeric_cols)
